# Getting Set Up

Running PyROA you have a two options:

1. Install PyROA package and use it directly
2. Install pyPETaL package, which is a "wrapper/pipeline" code that simplifies inputs, use other lag detection software, at the cost of a bit less control of inputs

Both are great options, I've tested both and gotten similar results, mainly it's only annoying to use pyPETaL if you want the output plots to be anything but their default, which is the biggest draw back since you may want to present your results in more than one way. That _some_ priors are calculated for you, which should probably be fine, but using the PyROA package give more control. (pyPETaL may be easier to learn on though).

### PyROA Install
Straight foward installation, just run the following command in your terminal:
```
    pip install pyroa
```
Here's the [PyROA GitHub Link](https://github.com/FergusDonnan/PyROA) which has their publication for in depth detail, and further examples/documentation

### pyPETaL Install
Run the following in terminal
```
    pip install pypetal
```
I'll be writing this tutorial for PyROA, so if you choose to use pyPETaL, just follow this up until the part I give example code. 
Then, just follow the [pyPETaL example usage of PyROA at this link](https://pypetal.readthedocs.io/en/latest/pyroa_toc.html)

### Issues After Install?
Should be straightforward, but the only issues I've had are missing packages PyROA uses. Just install the package in the error code if this is the case 
```
    pip install <package-name-here>
```
Otherwise, PyROA may require a specific version of numpy? If anything look at the issues tab on [their github](https://github.com/FergusDonnan/PyROA) for this.
Installing different version of numpy is as follows
THOUGH! You may want to use a seperate conda enviornemt BEFOREHAND (google how to make seperate conda enviornment p straight forward super useful), that way if you use other packages that need your current numpy version you don't mess that up.
```
    pip install numpy==<version-number-here>
```

# PyROA Basics
PyROA uses a running optimal average (ROA) model in order to simultaneously fit your input lightcurves. This is essentially exactly as it sounds, an optimally calculated average of your input photometry, which prioritizes this average calculation to epochs more "local" in time/MJD. The specifics of this model can be found in [Donnan et al. 2021](https://ui.adsabs.harvard.edu/abs/2021arXiv210712318D/abstract), but you don't need to know too much more other than this model fits all of your light curves simulatneously! (powerful stuff) Here's the basic run down of the model fit parameters used in the fit:

## Parameters and Priors

| Fit Parameters   | Description | 
| -------- | ------- |
| $A_i$  | Fraction of the RMS flux (model/observed) for your $i$th input lightcurve |
| $B_i$ | Fraction of the mean flux (model/observed) for your $i$th input lightcurve |
| $\tau_i$   | Lag measured between your bluest and $i$th lightcurve model |
| $\sigma_i$   | *Optional* extra variance that can be added to your $i$th lightcurve model (helps with fitting!) |
| $\Delta$   | The "window" in $t$ which all your lightcurve models will prioritize when calculating the ROA  |



PyROA runs a built in markov chain monte carlo (MCMC) fitting your data with a preset amount of walkers, over an input number of iterations. 

### Priors
If you're unfamiliar with the term, Bayesian statistics models generally will require prior knowledge of your fit parameters to know where the model should search. These can come in the form of probability distributions of any shape where the user assumes these model parameters should be, but often (and in PyROA's case) we assume a uniform prior distribution. (i.e. between lower and upper bounds).

PyROA will want you to input a list of upper and lower bounds to build a prior, as shown below.


```
    priors = [[A_lower, A_upper], [B_lower, B_upper], [tau_lower, tau_upper],[delta_lower, delta_upper], [sig_lower, sig_upper]]
```

THIS is where YOU have the most control of your fits. More specifically...

* A and B probably should stick around [0.5, 2.0], though can be adjusted to be wider if needed.
* [tau_lower, tau_upper] should porbably be centered around 0, just use reasonable bounds for lags you'd expect.
* The Delta prior may be the most sensitive! Try to keep the upper bound around ~5 times your cadence, but play around with this one a LOT. Having a smaller window mean your lightcurve model fixate on finer changes 
* $\sigma$ will also need some playing around with. With more range, the model will be more flexible, which can help it accomidate differences in variability between lightcurves.

Here's a starting point :)
```
    priors = [[0.5, 2.0],[0.5, 2.0], [0.0, 20.0], [0.05, 5.0], [0.0, 10.0]]
```


## PyROA Inputs

Here is what the PyROA command looks like with it's basic inputs.

```
    fit = PyROA.Fit(datadir, objName, filters, priors, add_var=True, plot_corner=True, init_tau = init_tau, init_delta = init_delta, Nsamples=Nsamples, Nburnin=Nburn)
    
```

IMPORTANT!!!
The way data is input is quite picky so datadir, objName, and filter CAREFULLY. Your filenames should be "objName" + "filter", and be dat files with coloums in the order of; 1) MJD, 2) flux, 3) flux_error.

Input Descriptions:
| Input   | Description | 
| -------- | ------- |
| datadir  | File location to the folder of your lightcurves. In this folder should one dat file for each lightcurve. |
| objName | Your file names in datadir should be "objName" + "filter" so objName is the beginning bit of each file name. ex) "sdssrm101" in "sdssrm101g filter.dat"|
| filters   | list of str, which are your filters/emission feature names. Ex) filters = ["g filter", "r filter", "i filter"] where you have the files "sdssrm101g filter.dat" etc. |
| priors   | list of upper and lower bounds for each fit parameter (see prior description above) |
| add_var   | Here you can choose whether you want the ROA model to include $\sigma_i$ the extra variance. It usually fits better with it, but not always. |
| plot_corner   | Whether or not you want PyROA to save a pdf of the posterior fit parameters distributions |
| init_tau   | list if initial guesses for each lag. If you have $n$ lightcurves, the list should be $n-1$ length |
| init_delta   | Initial guess for delta. May just want to give center of prior range. |
| Nsamples   | Number of MCMC iterations. Try 10000, you can add more if you think the model needs more time to settle on good fit parameters |
| Nburnin   | PyROA will disregard the first "Nburnin" iterations of the MCMC. This is useful for complex models that need thousands of iterations to fit there higher number of parameters (4 * $n$ lightcurves + 1 params!!!)|


## PyROA Outputs

PyROA will store it's outputs in a couple files, which they save right to whatever your current working directory is (eww). These files are...

* A corner plot
* "samples.obj" - a pickle object containting the total unflattened samples
* "samples_flat.obj" - a pickle object containing the flattened samples with the "burn-in" removed
* "X_t.obj" - a pickle object containting the driving lightcurve. This is a list in the form [t, X, X_errs], where t is the time grid, X is the value of the driving lightcurve, and X_errs is the error envelope from the running optimal average.
* "Lightcurve_models.obj" - a pickle object containing the model for each lightcurve. This is a list of models where each model is an array with in the form [t, model, model_errs].

Pickle files are just a method of data storage, my code below will give you the way to simply extract and plot your results.

ALSO I suggest you make a new folder for organizing PyROA outputs, otherwise everytime it is run, you will simply continue to dump these files into your working directory, which the developers disregaurd for there users to simply build in a desired output directory (just throwing shade). The code below will also include a way to close these files and also move them to your desired output directory.


In [ ]:
#Packages you will need

# Enable inline plotting in notebook
%matplotlib inline
# Populate namespace with numerical python function library and matplotlib plotting library.
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

print(np.__version__)


from matplotlib import gridspec#
from scipy import interpolate
from astropy.io import ascii
from astropy import units as u
from scipy.stats import median_abs_deviation
import collections
from uncertainties import ufloat
import math
import PyROA
import shutil
import pickle



# Example

Much of this example code (data included) is riped from the [example on the PyROA github](https://github.com/FergusDonnan/PyROA/blob/main/examples/Example_Usage.ipynb). I've just added my own little tweaks for the data output pretty much.



In [ ]:
#User Inputs

datadir = "......../MockData/HighSN/" #route to High SN data (feel free to try low SN too)
objName="TestObj" #these should be formatted right? feel free to double check
filters=["1","2","3"] #they just named the filters numerically
init_tau = [5.0, 10.0] #initial guess for lags
priors = [[0.5, 2.0],[0.5, 2.0], [0.0, 20.0], [0.05, 5.0], [0.0, 10.0]] #A, B, tau, sigma, delta (prior ranges)



In [ ]:
#here's a little function I use to move my output PyROA files to outputdirectory I want

def move_pyroa_files(dir_to_move_to, dir_where_files_are):
    """Function that moves PyROA outputs to a directory of your choice.
    Args:
        dir_to_move_to (str): directory to move files to.
        dir_where_files_are (str): directory where the files get placed. Usually it’s the base directory that you open up in your code editor.
    """
    files_to_move = ['CornerPlot1.pdf', 'samples.obj', 'samples_flat.obj', 'X_t.obj', 'Lightcurve_models.obj']
    for file in files_to_move:
        shutil.move(dir_where_files_are + file, dir_to_move_to + file)

outputdir = #Output directory here for PyROA files
workdir = '' #probably leave as '', otherwise write whatever you use as working directory

In [ ]:
#Run PyROA (this may take ~20 min depending on quality of data and Nsamples)

fit = PyROA.Fit(datadir, objName, filters_scaled, priors, add_var=ADDvar, plot_corner=True, init_tau = init_tau, init_delta = init_delta, Nsamples=Nsamples, Nburnin=Nburn)

#Move PyROA Files (otherwise they overwrite eachother)
move_pyroa_files(outputdir, workdir)

# Walker Trace Plot
The PyROA github example doesn't have this and there is now easy way to actually make this without some annoying looking into the devcode so you can use mine :) I don't think you'll need to edit any of this, but lmk otherwise and feel free to skip if so.

In [ ]:
file = open(outputdir + "samples.obj",'rb')
samples = pickle.load(file)

row_labels = ['$A_i$', '$B_i$', '$\\tau_i$', '$\\sigma_i$']

col_labels = [i[0:1] for i in filters]
col_labels.append("$\\Delta$")

ncol = len(filters) + 1

fig, ax = plt.subplots(4, ncol, figsize = (5*ncol, 4), sharex=True)

for i in range(0, 4):
    ax[i,0].set_ylabel(row_labels[i], fontsize = 28)


ind=[[0,1,2], [3,4,5,6], [7,8,9,10], [11,12,13,14]]
if len(filters) == 3:
    ind=[[0,1,2], [3,4,5,6], [7,8,9,10]]

if ADDvar == False:
    ind=[[0,1], [2,3,4], [5,6,7], [8,9,10]]

print(ind) 
    
for i in range(0, ncol):
    
    for j in range(samples.shape[1]):
        
        
        if i == ncol-1:
            ax[0, i].plot(samples[:,j,-1], c='k', alpha=0.1)
            continue
        
        ax[0,i].plot(samples[:,j,ind[i][0]], color='k', alpha = 0.1)
        ax[1,i].plot(samples[:,j,ind[i][1]], color='k', alpha = 0.1)
        
        
            
        if ADDvar == True:
            ax[3, i].plot(samples[:,j,ind[i][-1]], color='k', alpha = 0.1)
            if i == 0:
                ax[2,i].plot( np.zeros( samples.shape[0] ), c='k', alpha=0.1 )
            else:
                ax[2,i].plot(samples[:,j,ind[i][2]], color='k', alpha = 0.1)
        else:
            ax[3, i].plot(np.zeros( samples.shape[0] ), c='k', alpha=0.1 )
            if i == 0:
                ax[2,i].plot( np.zeros( samples.shape[0] ), c='k', alpha=0.1 )
            else:
                ax[2,i].plot(samples[:,j,ind[i][-1]], color='k', alpha = 0.1)
        
    for k in range(0,4):
        if k == 0:
            ax[k,i].set_title(col_labels[i], fontsize=28)
        ax[k,i].axvline(Nburn, ls='--', color='r')
        

# Model Lag Plot
This is pretty much ripped from the github example with a few quality of life tweaks. I have a more complex version, as I believe this may break if you choose add_var = False.

In [ ]:
plt.rcParams.update({
    "font.family": "Sans",  
    "font.serif": ["DejaVu"],
"figure.figsize":[40,30],
"font.size": 30})  

file = open(outputdir + "samples_flat.obj",'rb')
samples_flat = pickle.load(file)
file = open(outputdir + "Lightcurve_models.obj",'rb')
models = pickle.load(file)


#Split samples into chunks, 4 per lightcurve i.e A, B, tau, sig
chunk_size=4
transpose_samples = np.transpose(samples_flat)
#Insert zero where tau_0 would be 
transpose_samples = np.insert(transpose_samples, [2], np.array([0.0]*len(transpose_samples[1])), axis=0)
samples_chunks = [transpose_samples[i:i + chunk_size] for i in range(0, len(transpose_samples), chunk_size)] 

##########################################################################
#Hugh wrote this part to display lags in the corner of the plots
lag_dict = {}
for i in range(0, len(filters)):
    tau_samples = samples_chunks[i][tauind]
    print(len(tau_samples))
    lag_perc = np.percentile(tau_samples, [16, 50, 84])
    if i == 0:
        lag_text = "$\\tau$=" + str(lag_perc[1])[0:4] + " days"
    else:
        lag_text = "$\\tau$=" + str(lag_perc[1])[0:4] + " days \n(+" + str(lag_perc[2]-lag_perc[1])[0:4] + " -" + str(lag_perc[1]-lag_perc[0])[0:4] + ")"
    lag_dict[str(filters[i])[0:1]] = lag_text
print(lag_dict)
##########################################################################

fig = plt.figure(5)
gs = fig.add_gridspec(len(filters), 2, hspace=0, wspace=0, width_ratios=[5, 1])
axs= gs.subplots(sharex='col')

band_colors=["royalblue", "darkcyan", "olivedrab", "#ff6f00", "#ef0000", "#610000"]

#Loop over lightcurves
filters=["1","2","3"]

data=[]
for i in range(len(filters)):
    #Read in data
    file = datadir + "TestObj_" + str(filters[i]) + ".dat"
    data.append(np.loadtxt(file))
    mjd = data[i][:,0]
    flux = data[i][:,1]
    err = data[i][:,2]    
    
    #Add extra variance
    sig = np.percentile(samples_chunks[i][-1], 50)
    err = np.sqrt(err**2 + sig**2)
    
    #Plot Data
    axs[i][0].errorbar(mjd, flux , yerr=err, ls='none', marker=".", color=band_colors[i], ms=20, elinewidth=5)
    #Plot Model
    t, m, errs = models[i]
    axs[i][0].plot(t,m, color="black", lw=3)
    axs[i][0].fill_between(t, m+errs, m-errs, alpha=0.5, color="black")
    axs[i][0].set_xlabel("Time")
    axs[i][0].set_ylabel("Flux")

    #Plot Time delay posterior distributions
    tau_samples = samples_chunks[i][2],
    axs[i][1].hist(tau_samples, color=band_colors[i], bins=50)
    axs[i][1].axvline(x = np.percentile(tau_samples, [16, 50, 84])[1], color="black")
    axs[i][1].axvline(x = np.percentile(tau_samples, [16, 50, 84])[0] , color="black", ls="--")
    axs[i][1].axvline(x = np.percentile(tau_samples, [16, 50, 84])[2], color="black",ls="--")
    axs[i][1].axvline(x = 0, color="black",ls="--")    
    axs[i][1].set_xlabel("Time Delay ")
    axs[0][0].set_title("Lightcurves")
    axs[0][1].set_title("Time Delay")

    #Writes the lag in the corner of PyROA plot
    axs[i][1].text(0.7,0.85, lag_dict[fs[i]], horizontalalignment='center', verticalalignment='center', transform = axs[i][1].transAxes)
    

for ax in axs.flat:
    ax.label_outer()    
